# Pseudo code for KNN classifier

In [43]:
!pip3 install pandas sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [44]:
import json
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [45]:
pro_rb_data_scores = pd.read_csv('../ProData/RB_career_stats_scored.csv')
college_rb_data = pd.read_csv('../CollegeData/all_rushing.csv')

## Iterate through list of matches and map college id to nfl id

In [46]:
with open('./NFL EDA/matches.json', 'r') as match_file:
  match_data = json.load(match_file)

In [47]:
player_id_tuple = [(match['nfl_id'], match['college_id']) for match in match_data]

nfl_id, college_id = list(zip(*player_id_tuple))
player_ids = pd.DataFrame.from_dict({
  'nfl_id': nfl_id,
  'college_id': college_id
}).set_index('nfl_id')

len(player_ids)

3865

## Reset pro data index to college player id

In [48]:
target = pro_rb_data_scores.set_index('player_id').join(player_ids, how='inner').set_index('college_id')
print(len(pro_rb_data_scores))
print(len(target))
target.head()

394
340


,season_count,career_score
college_id,,
ameer-abdullah-1,8,-0.155174
joseph-addai-1,6,0.962896
jay-ajayi-1,6,0.423211
cam-akers-1,1,0.296292
shaun-alexander-1,9,1.361225


## Split Scored Pro data into tiers for model training / testing

In [49]:
TIERS = 5

sorted_scores = target.sort_values(by='career_score').reset_index()
number_of_players = len(sorted_scores)
increment = number_of_players // TIERS
print(increment)
print(number_of_players)


prev_idx = 0
for tier_num, i in enumerate(range(0, number_of_players, increment)):
    print(tier_num)
    increment_end = prev_idx + increment
    if tier_num + 1 == TIERS:
        print('creating last group')
        sorted_scores.loc[prev_idx:, 'tier'] = tier_num
    else:
        print(f'getting players {prev_idx} - {increment_end}')
        sorted_scores.loc[prev_idx:increment_end, 'tier'] = tier_num

    prev_idx = i + increment
  
target = sorted_scores.set_index('college_id').drop(['season_count', 'career_score'], axis=1)

68
340
0
getting players 0 - 68
1
getting players 68 - 136
2
getting players 136 - 204
3
getting players 204 - 272
4
creating last group


## Clean College

In [50]:
players_to_keep = set(target.index.values).intersection(set(college_rb_data.playerId.values))


In [51]:
len(players_to_keep)

327

In [52]:
grouped_data = college_rb_data.groupby('playerId').sum().loc[players_to_keep]
college_feature_data = grouped_data
for feature in grouped_data:
    if feature == 'g':
        college_feature_data['g'] = grouped_data['g']
        continue
        
    college_feature_data[feature] = grouped_data[feature] / grouped_data['g']

/var/folders/ky/3trdlcm94d9dgq_70q3lbmjr0000gn/T/ipykernel_91689/3551830286.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grouped_data = college_rb_data.groupby('playerId').sum().loc[players_to_keep]


In [53]:
target = target.loc[players_to_keep]
target.sort_values(by='tier', ascending=False).head(50)

/var/folders/ky/3trdlcm94d9dgq_70q3lbmjr0000gn/T/ipykernel_91689/2461575306.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  target = target.loc[players_to_keep]


,tier
college_id,
cedric-benson-1,4.0
leonard-fournette-1,4.0
steven-jackson-3,4.0
jahvid-best-1,4.0
miles-sanders-1,4.0
james-conner-1,4.0
alvin-kamara-1,4.0
matt-forte-1,4.0
ryan-mathews-1,4.0


In [54]:
target[target.isna().any(axis=1)]

,tier
college_id,


## Train Model

### Split train and test data

In [55]:
features = college_feature_data.sort_index()
target = target.sort_index()
print(len(target), len(features))

327 327


In [56]:
# X -> features, y -> label
X = features
y = target['tier']
 
# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

### Train model

In [57]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6).fit(X_train, y_train)
 
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print(accuracy)
 
# creating a confusion matrix
knn_predictions = knn.predict(X_test)
cm = confusion_matrix(y_test, knn_predictions)

0.25609756097560976
